## General

In [1]:
import tensorflow as tf
import numpy as np
import random

word_vectors = {}

glove_size = 0
with open('data/glove.6B.100d.txt', 'r') as f:
    for line in f:
        glove_size+=1
print glove_size

review_size = 0
with open('data/reviews_Books_5.json', 'r') as f:
    for line in f:
        review_size+=1
print review_size

line_index = 0
with open('data/glove.6B.100d.txt', 'r') as f:
    for line in f:
        line_index+=1
        if line_index%100000==0:
            print "Reading word_vectors... "+str(line_index)+"/"+str(glove_size)
        line_list = line.split()
        word = line_list[0]
        vec = [float(v) for v in line_list[1:]]
        assert len(vec) == 100
        word_vectors[word] = vec
        
print len(word_vectors)

400000
8898041
Reading word_vectors... 100000/400000
Reading word_vectors... 200000/400000
Reading word_vectors... 300000/400000
Reading word_vectors... 400000/400000
400000


In [4]:
x_data = []
y_data = []

line_index = 0
with open('data/reviews_Books_5.json', 'r') as f:
    for line in f:
        line_index+=1
        if line_index%100000==0:
#             print "Reading word_vectors... "+str(line_index)+"/"+str(review_size)
            print "Valid reviews: "+str(len(y_data))
        if "reviewText" not in line or "overall" not in line:
            continue
        review_start = line.index("\"reviewText\":")+15
        review_end = line.index("\"overall\":")-3
        if review_end-review_start<50:
            continue
        review = line[review_start:review_end]
        overall = line[review_end+14:review_end+15]
        
        for i in xrange(len(review)):
            if not review[i].isalpha():
                review = review[:i]+" "+review[i:]
        review = review.split()
        final_vec = np.zeros(100)
        n_skip = 0
        for word in review:
            word_l = word.lower()
            if word_l not in word_vectors:
                n_skip += 1
            else:
                final_vec+=np.array(word_vectors[word_l])
        if n_skip>0.25*len(review):
            continue
        else:
            final_vec/=float(len(review)-n_skip)
            overall = int(overall)
            x_data.append(final_vec)
            y_data.append(overall)

# print "Reading word_vectors... "+str(line_index)+"/"+str(review_size)
print "Valid reviews: "+str(len(y_data))
x_data = np.array(x_data)
y_data = np.array(y_data)
y_onehot = np.zeros((len(y_data), 5))
y_onehot[np.arange(len(y_data)), y_data-1] = 1

print x_data.shape
print y_data.shape
print y_onehot.shape
print "-------------"
print x_data[:5]
print y_onehot[:5]

Valid reviews: 48933
Valid reviews: 97870
Valid reviews: 146819
Valid reviews: 195769
Valid reviews: 244603
Valid reviews: 293552
Valid reviews: 342548


KeyboardInterrupt: 

In [ ]:
y_stats = np.sum(y_onehot, axis = 0)
print y_stats
num_in_class = min(y_stats)
new_x = []
new_y = []
new_y_onehot = []
nums = [0,0,0,0,0]
for i in xrange(len(y_data)):
    index = y_data[i]
    if nums[index]<num_in_class:
        new_x.append(x_data[i])
        new_y.append(y_data[i])
        new_y_onehot.append(y_onehot[i])
x_data = new_x
y_data = new_y
y_onehot = new_y_onehot
x_data = np.array(x_data)
y_data = np.array(y_data)
y_onehot = np.array(y_onehot)
y_stats = np.sum(y_onehot, axis = 0)
print y_stats
np.save("x_data", x_data)
np.save("y_onehot", y_onehot)
np.save("y_data", y_data)

In [ ]:
LR = 0.0001
epsilon = 0.000001

x_data = np.load('x_data.npy')
y_onehot = np.load('y_onehot.npy')
y_data = np.load('y_data.npy')

x_train = x_data[:400000]
y_train = y_onehot[:400000]

keep_prob = tf.placeholder(tf.float32)
xs = tf.placeholder(tf.float32, [None, 100])
ys = tf.placeholder(tf.float32, [None, 5])

def add_layer(inputs, in_size, out_size, activation_function=None):
    Weights = tf.Variable(tf.random_normal([in_size, out_size], mean=0., stddev=1.))
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, )
    Wx_plus_b = tf.matmul(inputs, Weights) + biases
    Wx_plus_b = tf.nn.dropout(Wx_plus_b, keep_prob)
    fc_mean, fc_var = tf.nn.moments(Wx_plus_b,axes=[0],)
    scale = tf.Variable(tf.ones([out_size]))
    shift = tf.Variable(tf.zeros([out_size]))
    Wx_plus_b = tf.nn.batch_normalization(Wx_plus_b, fc_mean, fc_var, shift, scale, epsilon)
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    return outputs

l1 = add_layer(xs, 100, 200, activation_function=tf.nn.relu)
l2 = add_layer(l1, 200, 200, activation_function=tf.nn.relu)
output = add_layer(l2, 200, 5, activation_function=tf.nn.relu)
output = tf.nn.softmax(output)

loss = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(output), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(LR).minimize(loss)
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    _xs = sess.run(xs, feed_dict={xs: x_train[:100], ys: y_train[:100], keep_prob: 1})
    _l1 = sess.run(l1, feed_dict={xs: x_train[:100], ys: y_train[:100], keep_prob: 1})
    output_ = sess.run(output, feed_dict={xs: x_train[:100], ys: y_train[:100], keep_prob: 1})
    loss_value = sess.run(loss, feed_dict={xs: x_train[:100], ys: y_train[:100], keep_prob: 1})
    print _xs
    print _l1
    print loss_value
    print output_
    print "------------------------"
    sess.run(train_step, feed_dict={xs: x_train[j*100:(j+1)*100], ys: y_train[j*100:(j+1)*100], keep_prob: 0.5})
    _xs = sess.run(xs, feed_dict={xs: x_train[:100], ys: y_train[:100], keep_prob: 1})
    _l1 = sess.run(l1, feed_dict={xs: x_train[:100], ys: y_train[:100], keep_prob: 1})
    output_ = sess.run(output, feed_dict={xs: x_train[:100], ys: y_train[:100], keep_prob: 1})
    loss_value = sess.run(loss, feed_dict={xs: x_train[:100], ys: y_train[:100], keep_prob: 1})
    print _xs
    print _l1
    print loss_value
    print output_

In [ ]:
LR = 0.2

sess = tf.Session()
sess.run(init)
for i in xrange(30):
    for j in xrange(4000):
        sess.run(train_step, feed_dict={xs: x_train[j*100:(j+1)*100], ys: y_train[j*100:(j+1)*100], keep_prob: 0.5})
    ys_ = sess.run(ys, feed_dict={xs: x_train[:100], ys: y_train[:100], keep_prob: 1})
    output_ = sess.run(output, feed_dict={xs: x_train, ys: y_train, keep_prob: 1})
    loss_value = sess.run(loss, feed_dict={xs: x_train, ys: y_train, keep_prob: 1})
#         print "Round "+str(i)+": Batch "+str(j)+" -- "+str(ys_)
#         print "Round "+str(i)+": Batch "+str(j)+" -- "+str(output_)
    pred = np.argmax(output_, axis = 1)
    a = 0
    for ind in xrange(len(pred)):
        if pred[ind] == y_data[ind]:
            a+=1
    correct_ratio = float(a)/len(pred)
    print "Round "+str(i)+" loss: "+str(loss_value)+", correct_ratio "+" -- "+str(correct_ratio)
    LR*=0.9

In [ ]:
x_test = x_data[400000:]
y_test = y_data[400000:]
yh_test = y_onehot[400000:]
print x_test.shape
print y_test.shape
print yh_test.shape
output_ = sess.run(output, feed_dict={xs: x_test, ys: yh_test, keep_prob: 1})
pred = np.argmax(output_, axis = 1)
a = 0
for ind in xrange(len(pred)):
    if pred[ind] == y_test[ind]:
        a+=1
correct_ratio = float(a)/len(pred)
print "correct_ratio "+" -- "+str(correct_ratio)

In [ ]:
x_data = np.load('x_data.npy')
y_onehot = np.load('y_onehot.npy')
y_data = np.load('y_data.npy')

x_train = x_data[:400000]
y_train = y_onehot[:400000]
x_test = x_data[400000:]
y_test = y_data[400000:]
yh_test = y_onehot[400000:]

print x_train.shape
print y_train.shape
print "------------"
print x_test.shape
print y_test.shape
print yh_test.shape

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=1)
print "training..."
neigh.fit(x_train[:5000], y_data[:5000])
pred = neigh.predict(x_test)
print pred.shape
a = 0
for ind in xrange(len(pred)):
    if pred[ind] == y_test[ind]:
        a+=1
correct_ratio = float(a)/len(pred)
print "correct_ratio "+" -- "+str(correct_ratio)

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=5)
print "training..."
neigh.fit(x_train[:5000], y_data[:5000])
pred = neigh.predict(x_test)
print pred.shape
a = 0
for ind in xrange(len(pred)):
    if pred[ind] == y_test[ind]:
        a+=1
correct_ratio = float(a)/len(pred)
print "correct_ratio "+" -- "+str(correct_ratio)

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=10)
print "training..."
neigh.fit(x_train[:5000], y_data[:5000])
pred = neigh.predict(x_test)
print pred.shape
a = 0
for ind in xrange(len(pred)):
    if pred[ind] == y_test[ind]:
        a+=1
correct_ratio = float(a)/len(pred)
print "correct_ratio "+" -- "+str(correct_ratio)

In [ ]:
from sklearn import svm
lin_clf = svm.LinearSVC()
lin_clf.fit(x_train[:10000], y_data[:10000])
pred = lin_clf.decision_function(x_test)
pred = np.argmax(pred, axis = 1)
print pred.shape
a = 0
for ind in xrange(len(pred)):
    if pred[ind] == y_test[ind]:
        a+=1
correct_ratio = float(a)/len(pred)
print "correct_ratio "+" -- "+str(correct_ratio)

## LSTM

In [ ]:
x_data = []
y_data = []

padding_vec = np.zeros(100)

line_index = 0
with open('data/reviews_Books_5.json', 'r') as f:
    for line in f:
        line_index+=1
        if line_index%100000==0:
#             print "Reading word_vectors... "+str(line_index)+"/"+str(review_size)
            print "Valid reviews: "+str(len(y_data))
        if "reviewText" not in line or "overall" not in line:
            continue
        review_start = line.index("\"reviewText\":")+15
        review_end = line.index("\"overall\":")-3
        if review_end-review_start<50:
            continue
        review = line[review_start:review_end]
        overall = line[review_end+14:review_end+15]
        
        for i in xrange(len(review)):
            if not review[i].isalpha():
                review = review[:i]+" "+review[i:]
        review = review.split()
        
        word_sequence = []
        for i in xrange(50):
            if i>=len(review):
                word_vec = padding_vec
            else:
                word = review[i].lower()
                if word in word_vectors:
                    word_vec = np.array(word_vectors[word])
                else:
                    word_vec = np.random.rand(100)
            word_sequence.append(word_vec)
        x_data.append(word_sequence)
        overall = int(overall)
        y_data.append(overall)

x_data = np.array(x_data)
y_data = np.array(y_data)
x_data = np.transpose(x_data,(1,0,2))
print x_data.shape
print y_data.shape
y_onehot = np.zeros((len(y_data), 5))
y_onehot[np.arange(len(y_data)), y_data-1] = 1

In [ ]:
y_stats = np.sum(y_onehot, axis = 0)
print y_stats
num_in_class = min(y_stats)
new_x = []
new_y = []
new_y_onehot = []
nums = [0,0,0,0,0]
for i in xrange(len(y_data)):
    index = y_data[i]
    if nums[index]<num_in_class:
        new_x.append(x_data[i])
        new_y.append(y_data[i])
        new_y_onehot.append(y_onehot[i])
x_data = new_x
y_data = new_y
y_onehot = new_y_onehot
x_data = np.array(x_data)
y_data = np.array(y_data)
y_onehot = np.array(y_onehot)
y_stats = np.sum(y_onehot, axis = 0)
print y_stats

In [ ]:
np.save("x_data_lstm", x_data)
np.save("y_onehot_lstm", y_onehot)
np.save("y_data_lstm", y_data)

In [ ]:
from tensorflow.contrib import rnn
import tensorflow as tf
import numpy as np
import random

x_data = np.load('x_data_lstm.npy')
y_onehot = np.load('y_onehot_lstm.npy')
y_data = np.load('y_data_lstm.npy')

x_train = x_data[:,:90000,:]
y_train = y_onehot[:90000]
x_test = x_data[:,90000:,:]
y_test = y_onehot[90000:]

LR = 0.1
alpha = 0.99
epsilon = 0.000001
hidden = 64

keep_prob = tf.placeholder(tf.float32)
xs = tf.placeholder(tf.float32, [50, None, 100])
ys = tf.placeholder(tf.float32, [None, 5])

def add_layer(inputs, in_size, out_size, activation_function=None):
    Weights = tf.Variable(tf.random_normal([in_size, out_size], mean=0., stddev=1.))
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, )
    Wx_plus_b = tf.matmul(inputs, Weights) + biases
    Wx_plus_b = tf.nn.dropout(Wx_plus_b, keep_prob)
    fc_mean, fc_var = tf.nn.moments(Wx_plus_b,axes=[0],)
    scale = tf.Variable(tf.ones([out_size]))
    shift = tf.Variable(tf.zeros([out_size]))
    Wx_plus_b = tf.nn.batch_normalization(Wx_plus_b, fc_mean, fc_var, shift, scale, epsilon)
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    return outputs
fw_cells = [rnn.BasicLSTMCell(hidden,state_is_tuple=True)]
bw_cells = [rnn.BasicLSTMCell(hidden,state_is_tuple=True)]
# xs_unstack = tf.unstack(xs,axis=0)
output_bi, output_state_fw, output_state_bw = rnn.stack_bidirectional_dynamic_rnn(fw_cells,bw_cells,xs,dtype=tf.float32,time_major=True)
features = tf.concat([output_state_fw[0][0], output_state_fw[0][1], output_state_bw[0][0], output_state_bw[0][1]], 1)

l1 = add_layer(features, 256, 500, activation_function=tf.nn.relu)
l2 = add_layer(l1, 500, 500, activation_function=tf.nn.relu)
l3 = add_layer(l2, 500, 5, activation_function=tf.nn.relu)
output = tf.nn.softmax(l3)

loss = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(output), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(LR).minimize(loss)
init = tf.global_variables_initializer()

"""
with tf.Session() as sess:
    sess.run(init)
    _xs = sess.run(xs, feed_dict={xs: x_train[:, :100, :], ys: y_train[:100], keep_prob: 1})
    _output_fw = sess.run(output_state_fw[0], feed_dict={xs: x_train[:, :100, :], ys: y_train[:100], keep_prob: 1})
    _output_bw = sess.run(output_state_bw[0], feed_dict={xs: x_train[:, :100, :], ys: y_train[:100], keep_prob: 1})
    print _xs.shape
    print _output_fw[0].shape
    print _output_fw[1].shape
    print _output_bw[0].shape
    print _output_bw[1].shape
    _output = sess.run(output_bi, feed_dict={xs: x_train[:, :100, :], ys: y_train[:100], keep_prob: 1})
    print type(_output)
    print _output.shape
    _feature = sess.run(features, feed_dict={xs: x_train[:, :100, :], ys: y_train[:100], keep_prob: 1})
    print _feature.shape
"""
print "Finished initialization"

In [ ]:
LR = 0.1
alpha = 0.99
epsilon = 0.000001
hidden = 64

with tf.Session() as sess:
    sess.run(init)
    for i in xrange(30):
        for j in xrange(900):
            sess.run(train_step, feed_dict={xs: x_train[:, j*100:(j+1)*100, :], ys: y_train[j*100:(j+1)*100, :], keep_prob: 0.5})
            if (j+1)%100 == 0:
                print "training... "+str(j+1)+"/900"
        _output = sess.run(output, feed_dict={xs: x_train[:, :100, :], ys: y_train[:100], keep_prob: 1})
        print _output.shape
        print _output
#         output_ = sess.run(output, feed_dict={xs: x_train, ys: y_train, keep_prob: 1})
#         pred = np.argmax(output_, axis = 1)
#         a = 0
#         for ind in xrange(len(pred)):
#             if pred[ind] == y_data[ind]:
#                 a+=1
#         correct_ratio = float(a)/len(pred)
#         print "Round "+str(i)+", correct_ratio "+" -- "+str(correct_ratio)
        LR*=alpha